# Polarization sensitivity

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

import logging
import math
from functools import reduce

import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from IPython.display import Math, display
from matplotlib import cm
from matplotlib.axes import Axes
from matplotlib.collections import LineCollection
from matplotlib.colors import LogNorm
from tensorwaves.function.sympy import create_parametrized_function
from tensorwaves.interface import DataSample
from tqdm.auto import tqdm

from polarization import formulate_polarization
from polarization.data import create_data_transformer, generate_meshgrid_sample
from polarization.function import compute_sub_function
from polarization.io import mute_jax_warnings, perform_cached_doit
from polarization.lhcb import (
    flip_production_coupling_signs,
    load_model_builder,
    load_model_parameters,
)
from polarization.lhcb.particle import load_particles
from polarization.plot import get_contour_line, stylize_contour, use_mpl_latex_fonts

logging.getLogger("polarization.function").setLevel(logging.INFO)
mute_jax_warnings()

Final state IDs:
1. $p$
2. $\pi^+$
3. $K^-$

Sub-system definitions:
1. $K^{**} \to \pi^+ K^-$
2. $\Lambda^{**} \to p K^-$
3. $\Delta^{**} \to p \pi^+$

In [ ]:
model_choice = 0
model_file = "../data/model-definitions.yaml"
particles = load_particles("../data/particle-definitions.yaml")
amplitude_builder = load_model_builder(model_file, particles, model_choice)
imported_parameter_values = load_model_parameters(
    model_file, amplitude_builder.decay, model_choice
)
models = {}
for reference_subsystem in [1, 2, 3]:
    models[reference_subsystem] = amplitude_builder.formulate(
        reference_subsystem, cleanup_summations=True
    )
    models[reference_subsystem].parameter_defaults.update(imported_parameter_values)
del reference_subsystem

models[2] = flip_production_coupling_signs(models[2], subsystem_names=["K", "L"])
models[3] = flip_production_coupling_signs(models[3], subsystem_names=["K", "D"])

DECAY = models[1].decay
FINAL_STATE = {
    1: "p",
    2: R"\pi^+",
    3: "K^-",
}

## SymPy expressions

In [ ]:
unfolded_polarization_exprs = {}
unfolded_intensity_expr = {}
for i, model in tqdm(models.items(), "Unfolding expressions"):
    reference_subsystem = i
    polarization_exprs = formulate_polarization(
        amplitude_builder, reference_subsystem
    )
    unfolded_polarization_exprs[i] = [
        perform_cached_doit(expr.doit().xreplace(model.amplitudes))
        for expr in tqdm(polarization_exprs, leave=False)
    ]
    unfolded_intensity_expr[i] = perform_cached_doit(model.full_expression)
del i, polarization_exprs, reference_subsystem

## Definition of free parameters

In [ ]:
polarization_funcs = {}
intensity_func = {}
for i, model in tqdm(models.items(), "Lambdifying to JAX"):
    production_couplings = {
        symbol: value
        for symbol, value in model.parameter_defaults.items()
        if isinstance(symbol, sp.Indexed)
        if "production" in str(symbol)
    }
    fixed_parameters = {
        symbol: value
        for symbol, value in model.parameter_defaults.items()
        if symbol not in production_couplings
    }
    polarization_funcs[i] = [
        create_parametrized_function(
            expr.xreplace(fixed_parameters),
            parameters=production_couplings,
            backend="jax",
        )
        for expr in tqdm(unfolded_polarization_exprs[i], leave=False)
    ]
    intensity_func[i] = create_parametrized_function(
        unfolded_intensity_expr[i].xreplace(fixed_parameters),
        parameters=production_couplings,
        backend="jax",
    )

del model
del fixed_parameters, production_couplings

## Polarization distributions

In [ ]:
data_sample = generate_meshgrid_sample(DECAY, resolution=400)
X = data_sample["sigma1"]
Y = data_sample["sigma2"]
for model in models.values():
    transformer = create_data_transformer(model)
    data_sample.update(transformer(data_sample))
del model, transformer

### Average polarization values

In [ ]:
def render_mean(array, weights, plus=True):
    array = array.real
    mean = compute_weighted_average(array, weights)
    variance = compute_weighted_average((array - mean) ** 2, weights)
    std = jnp.sqrt(variance)
    mean = f"{mean:.3f}"
    std = f"{std:.3f}"
    if plus and float(mean) > 0:
        mean = f"+{mean}"
    return Rf"{mean} \pm {std}"


def compute_weighted_average(values, weights):
    return jnp.nansum(values * weights) / jnp.nansum(weights)


def create_average_polarization_table(reference_subsystem: int) -> None:
    latex = R"\begin{array}{cccc}" + "\n"
    latex += (
        R"& \bar{|\alpha|} & \bar\alpha_x & \bar\alpha_y & \bar\alpha_z \\" + "\n"
    )
    weights = intensity_func[1](data_sample)
    for label, identifier in zip(subsystem_labels, subsystem_identifiers):
        latex += f"  {label} & "
        x, y, z = (
            compute_sub_function(
                polarization_funcs[reference_subsystem][xyz],
                data_sample,
                [identifier],
            )
            for xyz in range(3)
        )
        alpha_abs = jnp.sqrt(x**2 + y**2 + z**2)
        latex += render_mean(alpha_abs, weights, plus=False) + " & "
        latex += " & ".join(render_mean(i, weights) for i in [x, y, z])
        latex += R" \\" + "\n"
    latex += R"\end{array}"
    display(Math(latex))


subsystem_identifiers = ["K", "L", "D"]
subsystem_labels = ["K^{**}", R"\Lambda^{**}", R"\Delta^{**}"]

**Reference sub-system 1** ($K^{**} \to \pi^+ K^-$):

In [ ]:
create_average_polarization_table(1)

**Reference sub-system 2** ($\Lambda^{**} \to p K^-$):

In [ ]:
create_average_polarization_table(2)

**Reference sub-system 3** ($\Delta^{**} \to p \pi^+$):

In [ ]:
create_average_polarization_table(3)

### Dominant contributions

In [ ]:
%config InlineBackend.figure_formats = ['png']

In [ ]:
def create_dominant_region_contours(
    decay, data_sample: DataSample, threshold: float
) -> dict[str, jnp.ndarray]:
    I_tot = intensity_func[1](data_sample)
    resonance_names = [chain.resonance.name for chain in decay.chains]
    region_filters = {}
    progress_bar = tqdm(
        desc="Computing dominant region contours",
        total=len(resonance_names),
    )
    for resonance_name in resonance_names:
        progress_bar.postfix = resonance_name
        regex_filter = resonance_name.replace("(", r"\(").replace(")", r"\)")
        I_sub = compute_sub_function(intensity_func[1], data_sample, [regex_filter])
        ratio = I_sub / I_tot
        selection = jnp.select(
            [jnp.isnan(ratio), ratio < threshold, True],
            [0, 0, 1],
        )
        progress_bar.update()
        if jnp.all(selection == 0):
            continue
        region_filters[resonance_name] = selection
    contour_arrays = {}
    for contour_level, subsystem in enumerate(["K", "L", "D"], 1):
        contour_array = reduce(
            jnp.bitwise_or,
            (a for k, a in region_filters.items() if k.startswith(subsystem)),
        )
        contour_array *= contour_level
        contour_arrays[subsystem] = contour_array
    return contour_arrays


def indicate_dominant_regions(
    contour_arrays, ax: Axes, selected_subsystems=None
) -> dict[str, LineCollection]:
    if selected_subsystems is None:
        selected_subsystems = {"K", "L", "D"}
    selected_subsystems = set(selected_subsystems)
    colors = dict(K="red", L="blue", D="green")
    labels = dict(K="K^{**}", L=R"\Lambda^{**}", D=R"\Delta^{**}")
    legend_elements = {}
    for subsystem, Z in contour_arrays.items():
        if subsystem not in selected_subsystems:
            continue
        contour_set = ax.contour(X, Y, Z, colors="none")
        stylize_contour(
            contour_set,
            edgecolor=colors[subsystem],
            linewidth=0.5,
            label=f"${labels[subsystem]}$",
        )
        line_collection = get_contour_line(contour_set)
        legend_elements[subsystem] = line_collection
    return legend_elements

In [ ]:
%%time
nrows = 4
ncols = 5
scale = 3.0
aspect_ratio = 1.05
plt.rcdefaults()
use_mpl_latex_fonts()
plt.rc("font", size=15)
fig, axes = plt.subplots(
    dpi=200,
    figsize=scale * np.array([ncols, aspect_ratio * nrows]),
    gridspec_kw={"width_ratios": (ncols - 1) * [1] + [1.24]},
    ncols=ncols,
    nrows=nrows,
    sharex=True,
    sharey=True,
)
plt.subplots_adjust(wspace=0.05)

s1_label = R"$m^2\left(K\pi\right)$ [GeV$/c^2]$"
s2_label = R"$m^2\left(pK\right)$ [GeV$/c^2]$"
for subsystem in range(nrows):
    for i in range(ncols):
        ax = axes[subsystem, i]
        if i == 0:
            alpha_str = R"I_\mathrm{tot}"
        elif i == 1:
            alpha_str = R"|\alpha|"
        else:
            xyz = i - 2
            alpha_str = Rf"\alpha_{'xyz'[xyz]}"
        title = alpha_str
        if subsystem > 0:
            label = subsystem_labels[subsystem - 1]
            title = Rf"{title}\left({label}\right)"
        ax.set_title(f"${title}$")
        if ax is axes[-1, i]:
            ax.set_xlabel(s1_label)
        if i == 0:
            ax.set_ylabel(s2_label)

intensity_arrays = []
polarization_arrays = []
for subsystem in range(nrows):
    # alpha_xyz distributions
    alpha_xyz_arrays = []
    for i in range(2, ncols):
        xyz = i - 2
        if subsystem == 0:
            z_values = polarization_funcs[1][xyz](data_sample)
            polarization_arrays.append(z_values)
        else:
            identifier = subsystem_identifiers[subsystem - 1]
            z_values = compute_sub_function(
                polarization_funcs[1][xyz], data_sample, identifier
            )
        z_values = np.real(z_values)
        alpha_xyz_arrays.append(z_values)
        mesh = axes[subsystem, i].pcolormesh(X, Y, z_values, cmap=cm.coolwarm)
        mesh.set_clim(vmin=-1, vmax=+1)
        if xyz == 2:
            c_bar = fig.colorbar(mesh, ax=axes[subsystem, i])
            c_bar.set_ticks([-1, 0, +1])
            c_bar.set_ticklabels(["-1", "0", "+1"])
    # absolute value of alpha_xyz vector
    alpha_abs = np.sqrt(np.sum(np.array(alpha_xyz_arrays) ** 2, axis=0))
    mesh = axes[subsystem, 1].pcolormesh(X, Y, alpha_abs, cmap=cm.coolwarm)
    mesh.set_clim(vmin=-1, vmax=+1)
    # total intensity
    if subsystem == 0:
        z_values = intensity_func[1](data_sample)
    else:
        identifier = subsystem_identifiers[subsystem - 1]
        z_values = compute_sub_function(intensity_func[1], data_sample, identifier)
    intensity_arrays.append(z_values)
    axes[subsystem, 0].pcolormesh(X, Y, z_values, norm=LogNorm())

threshold = 0.7
contour_arrays = create_dominant_region_contours(DECAY, data_sample, threshold)

for ax in axes[0]:
    legend_elements = indicate_dominant_regions(contour_arrays, ax)
    if ax is axes[0, -1]:
        leg = ax.legend(
            handles=legend_elements.values(),
            title=Rf"$>{100*threshold:.0f}\%$",
            bbox_to_anchor=(0.9, 0.88, 1.0, 0.1),
            framealpha=1,
        )

for subsystem, ax_row in zip(["K", "L", "D"], axes[1:]):
    for ax in ax_row:
        indicate_dominant_regions(
            contour_arrays, ax, selected_subsystems=[subsystem]
        )

plt.show()

In [ ]:
plt.rcdefaults()
use_mpl_latex_fonts()
plt.rc("font", size=16)
fig, axes = plt.subplots(
    dpi=200,
    figsize=(13, 5),
    gridspec_kw={"width_ratios": [1, 1, 1.2]},
    ncols=3,
    sharey=True,
    tight_layout=True,
)
axes[0].set_ylabel(s2_label)
I_times_alpha = jnp.array(
    [array * intensity_arrays[0] for array in polarization_arrays]
)
global_min_max = float(jnp.nanmax(jnp.abs(I_times_alpha)))
for ax, z_values, xyz in zip(axes, I_times_alpha, "xyz"):
    ax.set_title(Rf"$\alpha_{xyz} \cdot I$")
    ax.set_xlabel(s1_label)
    mesh = ax.pcolormesh(X, Y, np.real(z_values), cmap=cm.RdYlGn_r)
    mesh.set_clim(vmin=-global_min_max, vmax=global_min_max)
    if ax is axes[-1]:
        fig.colorbar(mesh, ax=ax, pad=0.02)
plt.show()

### Total polarization vector field

In [ ]:
%config InlineBackend.figure_formats = ['svg']

In [ ]:
def plot_polarization_field(polarization_arrays, ax, strides=14, cmap=cm.RdYlGn_r):
    alpha_abs = jnp.sqrt(jnp.sum(polarization_arrays**2, axis=0))
    mesh = ax.quiver(
        X[::strides, ::strides],
        Y[::strides, ::strides],
        np.real(polarization_arrays[2][::strides, ::strides]),
        np.real(polarization_arrays[0][::strides, ::strides]),
        np.real(alpha_abs[::strides, ::strides]),
        cmap=cmap,
    )
    mesh.set_clim(vmin=0, vmax=+1)
    return mesh


def plot_total_field(reference_subsystem: int) -> None:
    plt.rcdefaults()
    use_mpl_latex_fonts()
    plt.rc("font", size=18)
    fig, ax = plt.subplots(figsize=(8, 7), tight_layout=True)
    fig.suptitle(R"Total polarization field $\vec{\alpha}$", x=0.45, y=0.93)
    ax.set_title(f"(reference subsystem {reference_subsystem})")
    ax.set_xlabel(s1_label + R",$\quad\vec{\alpha}_z$")
    ax.set_ylabel(s2_label + R",$\quad\vec{\alpha}_x$")
    polarization_arrays = [
        func(data_sample) for func in polarization_funcs[reference_subsystem]
    ]
    mesh = plot_polarization_field(jnp.array(polarization_arrays), ax)
    color_bar = fig.colorbar(mesh, ax=ax, pad=0.01)
    color_bar.set_label(R"$\left|\vec{\alpha}\right|$")
    plt.show()


plot_total_field(reference_subsystem=1)

In [ ]:
plot_total_field(reference_subsystem=2)

In [ ]:
plot_total_field(reference_subsystem=3)

### Aligned vector fields per chain

In [ ]:
def to_regex(text: str) -> str:
    regex = text
    regex = regex.replace("(", R"\(")
    regex = regex.replace(")", R"\)")
    return regex


def plot_polarization_field_per_resonance(reference_subsystem: int) -> None:
    spectator = FINAL_STATE[reference_subsystem]
    subsystem_name = subsystem_identifiers[reference_subsystem - 1]
    subsystem_resonances = [
        chain.resonance
        for chain in DECAY.chains
        if chain.resonance.name.startswith(subsystem_name)
    ]
    ncols = 3
    nrows = math.ceil(len(subsystem_resonances) / ncols)
    fig, axes = plt.subplots(
        figsize={1: (13, 5), 2: (13, 9.5)}[nrows],
        gridspec_kw={"width_ratios": [1, 1, 1.13]},
        ncols=3,
        nrows=nrows,
        sharex=True,
        sharey=True,
        tight_layout=True,
    )
    fig.suptitle(
        f"Polarization field, aligned to ${spectator}$",
        y={1: 0.92, 2: 0.96}[nrows],
    )
    for i, (ax, resonance) in enumerate(zip(axes.flatten(), subsystem_resonances)):
        non_zero_couplings = [to_regex(resonance.name)]
        polarizations = [
            compute_sub_function(func, data_sample, non_zero_couplings)
            for func in polarization_funcs[reference_subsystem]
        ]
        polarizations = jnp.array(polarizations).real
        abs_alpha = jnp.sqrt(jnp.sum(polarizations**2, axis=0))
        mesh = plot_polarization_field(
            polarizations,
            ax=ax,
            cmap=cm.viridis,
            strides=22,
        )
        mean = jnp.nanmean(abs_alpha)
        std = jnp.nanstd(abs_alpha)

        ax.text(
            x=1.85,
            y=4.45,
            s=Rf"$\overline{{\left|\vec\alpha\right|}} = {mean:.3f} \pm {std:.3f}$",
            fontsize=12,
            horizontalalignment="right",
        )
        ax.set_title(f"${resonance.latex}$")
        if i % 3 == 2:
            color_bar = fig.colorbar(mesh, ax=ax, pad=0.01)
            color_bar.set_label(R"$\left|\vec{\alpha}\right|$")
    plt.show()


for reference_subsystem in tqdm([1, 2, 3]):
    plot_polarization_field_per_resonance(reference_subsystem)
    del reference_subsystem

In [ ]:
fig, axes = plt.subplots(
    figsize=(12, 4.9),
    gridspec_kw={"width_ratios": [1, 1, 1.14]},
    ncols=3,
    sharey=True,
    tight_layout=True,
)
fig.suptitle("Polarization field per sub-system", y=0.93)
items = zip(axes, [1, 2, 3], subsystem_identifiers, subsystem_labels)
for ax, reference_subsystem, subsystem_name, subsystem_label in items:
    non_zero_couplings = [subsystem_name]
    polarizations = [
        compute_sub_function(func, data_sample, non_zero_couplings)
        for func in polarization_funcs[reference_subsystem]
    ]
    polarizations = jnp.array(polarizations).real
    abs_alpha = jnp.sqrt(jnp.sum(polarizations**2, axis=0))
    mesh = plot_polarization_field(
        polarizations,
        ax=ax,
        cmap=cm.viridis,
        strides=18,
    )
    mean = jnp.nanmean(abs_alpha)
    std = jnp.nanstd(abs_alpha)

    ax.text(
        x=1.8,
        y=4.4,
        s=Rf"$\overline{{\left|\vec\alpha\right|}} = {mean:.3f} \pm {std:.3f}$",
        fontsize=12,
        horizontalalignment="right",
    )
    spectator = FINAL_STATE[reference_subsystem]
    ax.set_title(f"${subsystem_label}$ (aligned to ${spectator}$)")
    if ax is axes[-1]:
        color_bar = fig.colorbar(mesh, ax=ax, pad=0.01)
        color_bar.set_label(R"$\left|\vec{\alpha}\right|$")